# 14813 final project option1 (fifa data)

co_work of

hainengh,\
Haineng Huang
    
kaixins,\
Kaixin Song

## 1 Docker setup

this part has largely referenced to the work of Yiren Zhou <<yirenzho@andrew.cmu.edu>>, and uses the Docker file provided by Yiren Zhou, thanks very much.\
The oringinal work can be visited here: <<https://github.com/ML-Systems-and-Toolchains/course-project-option-1-sample-2>>

### 1.1 start the contaioner (for both windows and linux system)

1.Please ensure that Docker is available (including docker compose).\
2.download the code from github,unzip if needed.For Linux, we suggest to put the folder under /home/\
3.open a cmd window (Windows) or a terminal (Linux)\
4.`cd` to downloaded file location\
5.execute `docker compose up -d`,which will install all the necessary dependencies.\
6.after the container starts, get into workspace by `docker exec -it --user root cp /bin/bash`

### 1.2 change user password of PostgreSQL and create schema
1.start the postgresql server by `service postgresql start`\
2.get into postgresql workspace by `sudo -u postgres psql`\
3.execute the following two SQL statements to 1) change the password and 2) create a schema named "fifa":\
`ALTER USER postgres PASSWORD 'bigdata';`\
`CREATE SCHEMA fifa;`\
4.exit the shell by `\q`

### 1.3 open jupyter notebook and execute code for Task 1_2_3
1.start the Jupyter notebook within the container by `jupyter notebook --ip 0.0.0.0 --allow-root`\
2.you should see a URL like: `http://127.0.0.1:8888/?token=574bb631fc2e3c790dcf6c0317f9f3ae674cab80264f1707`\
3.copy the URL (better use the one with `127.0.0.1`) and open it in web broswer\
4.Open <Task_1_2_3.ipynb>, and execute the cells sequentially.

### 1.4 Exit and stop container

1.exiting the Docker container by `Ctrl + D`

2.execute the following to stop the container

```
docker compose down
```


## 2 Task-I:Build and populate necessary tables

### 2.1 Build and populate necessary tables
implement the code cells in the jupyter notebook for the step

In [1]:
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .getOrCreate()

from pyspark.sql import SQLContext
sc=spark.sparkContext
sqlContext = SQLContext(sc)

sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/29 04:08:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


<SparkContext master=local[*] appName=GenericAppName>

In [2]:
from pyspark import SparkFiles

df_15 = spark.read.csv("data/players_15.csv",header=True, inferSchema= True)
df_16 = spark.read.csv("data/players_16.csv",header=True, inferSchema= True)
df_17 = spark.read.csv("data/players_17.csv",header=True, inferSchema= True)
df_18 = spark.read.csv("data/players_18.csv",header=True, inferSchema= True)
df_19 = spark.read.csv("data/players_19.csv",header=True, inferSchema= True)
df_20 = spark.read.csv("data/players_20.csv",header=True, inferSchema= True)
df_21 = spark.read.csv("data/players_21.csv",header=True, inferSchema= True)
df_22 = spark.read.csv("data/players_22.csv",header=True, inferSchema= True)

In [3]:
df_15.show(1,vertical=True)

22/11/29 04:09:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24 00:00:00  
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | 241.0                
 club_name               

In [4]:
df_15.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: timestamp (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: timestamp (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable

In [5]:
from pyspark.sql import functions

df_15 = df_15.withColumn("year",functions.lit(2015))
df_16 = df_16.withColumn("year",functions.lit(2016))
df_17 = df_17.withColumn("year",functions.lit(2017))
df_18 = df_18.withColumn("year",functions.lit(2018))
df_19 = df_19.withColumn("year",functions.lit(2019))
df_20 = df_20.withColumn("year",functions.lit(2020))
df_21 = df_21.withColumn("year",functions.lit(2021))
df_22 = df_22.withColumn("year",functions.lit(2022))

In [6]:
from functools import reduce
def unite_dfs(df1, df2):
    print("df count is",df2.count())
    return df2.union(df1)

list_of_dfs = [df_15, df_16, df_17, df_18, df_19, df_20, df_21, df_22]
print("df count is",df_15.count())
united_df = reduce(unite_dfs, list_of_dfs)
print("united df count is",united_df.count())

df count is 16155
df count is 15623
df count is 17596
df count is 17954
df count is 18085
df count is 18483
df count is 18944
df count is 19239
united df count is 142079


In [7]:
from pyspark.sql.functions import monotonically_increasing_id,lit
df = united_df.select("*").withColumn("id", monotonically_increasing_id())

add a new column 'id' for the tabel, which is unique

## 2.2 write the pyspark dataframe to postgresql table
implement the code cells in the jupyter notebook for the step

In [8]:
db_properties={}
#update your db username
db_properties['username']="postgres"
#update your db password
db_properties['password']="bigdata"
#make sure you got the right port number here
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres?currentSchema=fifa"
#db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
#make sure you had the Postgres JAR file in the right location
db_properties['driver']="org.postgresql.Driver"
db_properties['table']= "fifa"


df.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

## 2.3 setting the constraint for table
(for both windows & linux)\
1.`cd` to docker file location\
2.get into workspace by `docker exec -it --user root cp /bin/bash`\
3.get into postgresql workspace by `sudo -u postgres psql`\
3.setting the constraint for table by following code\
```
ALTER TABLE fifa.fifa
  ADD CONSTRAINT tablename_pkey 
     PRIMARY KEY (id); 
```


## 2.4 (Attachment) Descriptions for features

 'sofifa_id',
     id of players in fifa
     
 'player_url',
 'short_name',
 'long_name',
     name and personal website url
     
 'player_positions',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
     basical informations for players 
     
 'club_team_id',
 'club_name',
 'league_name',
 'league_level',
 'club_position',
 'club_jersey_number',
 'club_loaned_from',
 'club_joined',
 'club_contract_valid_until',
 'nationality_id',
 'nationality_name',
 'nation_team_id',
 'nation_position',
 'nation_jersey_number',
     club and nation information
     
 'preferred_foot',
 'weak_foot',
 'skill_moves',
 'international_reputation',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
     information for personal 
     
 'player_tags',
     tag on players
     
 'player_traits',
     special traits of players
     
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_reactions',
 'movement_balance',
 'power_shot_power',
 'power_jumping',
 'power_stamina',
 'power_strength',
 'power_long_shots',
 'mentality_aggression',
 'mentality_interceptions',
 'mentality_positioning',
 'mentality_vision',
 'mentality_penalties',
 'mentality_composure',
 'defending_marking_awareness',
 'defending_standing_tackle',
 'defending_sliding_tackle',
 'goalkeeping_diving',
 'goalkeeping_handling',
 'goalkeeping_kicking',
 'goalkeeping_positioning',
 'goalkeeping_reflexes',
 'goalkeeping_speed',
     score/grades on performance
     
 'ls',
 'st',
 'rs',
 'lw',
 'lf',
 'cf',
 'rf',
 'rw',
 'lam',
 'cam',
 'ram',
 'lm',
 'lcm',
 'cm',
 'rcm',
 'rm',
 'lwb',
 'ldm',
 'cdm',
 'rdm',
 'rwb',
 'lb',
 'lcb',
 'cb',
 'rcb',
 'rb',
 'gk',
     score/grades for different positions on the playground
     
 'player_face_url',
 'club_logo_url',
 'club_flag_url',
 'nation_logo_url',
 'nation_flag_url',
     picture url for players and their club, nation
     
 'year'
     the year of csv files this row of data came from


## 3 Task-II: Conduct analytics on your dataset

### 3.1 read the data from postgresql 
implement the code cells in the jupyter notebook for the step

In [9]:
sqlContext = SQLContext(spark)

In [10]:
df = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

In [11]:
df.count()

142079

In [12]:
df.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: timestamp (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: timestamp (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable

### 3.2 Functions for analyze
implement the code cells in the jupyter notebook for the step

3.2.1 What are the X=5 clubs that have the highest number of players with contracts ending in 2023 (in 2022 data)?

In [13]:
import pyspark.sql.functions as F
from pyspark.sql.functions import desc

def find_top_clabs_contrasts_ending_2023(df,X):
    # 2022 data only
    df_22 = df.filter(df.year ==2022 )
    df_filterd = df_22.filter(df_22.club_name != 'None').groupBy("club_name").count().sort(F.col('count').desc()).head(5)
    df_new = spark.createDataFrame(df_filterd)
    #return df_filterd
    return df_new

In [14]:
find_top_clabs_contrasts_ending_2023(df,5).show()

+--------------------+-----+
|           club_name|count|
+--------------------+-----+
|RCD Espanyol de B...|   33|
|             Burnley|   33|
|    Newcastle United|   33|
|    RC Celta de Vigo|   33|
| Paris Saint-Germain|   33|
+--------------------+-----+



3.2.2 List the Y=5 clubs with highest average number of players that are older than 27 years across all years (i.e. calculate the number of players older than 27 years old for each club in each dataset, calculate the averages and list the Y clubs with highest averages).

In [15]:
# age

def find_top_clabs_age_over_27(df,Y):
    df_age27 = df.filter(df.age > 27 )
    df_filtered = df_age27.groupBy("club_name").mean("age").sort(F.col("avg(age)").desc()).head(Y)
    #df_filtered = df_age27.groupBy("club_name").mean("age")
    df_new = spark.createDataFrame(df_filtered)
    #return df_filtered
    return df_new

In [16]:
find_top_clabs_age_over_27(df,5).show()

+--------------------+------------------+
|           club_name|          avg(age)|
+--------------------+------------------+
|         Yokohama FC|  34.7037037037037|
|      Wexford Youths|              34.0|
|  Zamora Fútbol Club|33.857142857142854|
|Centro Atlético F...|              33.6|
|      CF Fuenlabrada| 33.54545454545455|
+--------------------+------------------+



3.2.3 What is the most frequent nation_position in the dataset for each year? (i.e.display the most frequent nation_position for 2015, 2016, etc.)

In [17]:
# nation_position

def find_most_nation_from2015to2022(df):
    years = ["2015","2016","2017","2018","2019","2020","2021","2022"]
    for y in years:
        df_year = df.filter(df.year == y)
        df_filtered = df_year.filter(df_year.nation_position != 'None').groupBy("nation_position").count().sort(F.col("count").desc()).head(1)
        print("year of data is ",y)
        print(spark.createDataFrame(df_filtered).show())
    return None

In [18]:
find_most_nation_from2015to2022(df)

year of data is  2015
+---------------+-----+
|nation_position|count|
+---------------+-----+
|            SUB|  564|
+---------------+-----+

None
year of data is  2016
+---------------+-----+
|nation_position|count|
+---------------+-----+
|            SUB|  511|
+---------------+-----+

None
year of data is  2017
+---------------+-----+
|nation_position|count|
+---------------+-----+
|            SUB|  564|
+---------------+-----+

None
year of data is  2018
+---------------+-----+
|nation_position|count|
+---------------+-----+
|            SUB|  600|
+---------------+-----+

None
year of data is  2019
+---------------+-----+
|nation_position|count|
+---------------+-----+
|            SUB|  576|
+---------------+-----+

None
year of data is  2020
+---------------+-----+
|nation_position|count|
+---------------+-----+
|            SUB|  588|
+---------------+-----+

None
year of data is  2021
+---------------+-----+
|nation_position|count|
+---------------+-----+
|            SUB| 

## 4 Task-III: Machine Learning Modeling

### 4.1 read the data from postgresql 
implement the code cells in the jupyter notebook for the step

In [19]:
df = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

In [20]:
df.show(1, vertical=True)

-RECORD 0-------------------------------------------
 sofifa_id                   | 244554               
 player_url                  | https://sofifa.co... 
 short_name                  | J. Dodd              
 long_name                   | James Dodd           
 player_positions            | CM                   
 overall                     | 53                   
 potential                   | 66                   
 value_eur                   | 100000.0             
 wage_eur                    | 700.0                
 age                         | 18                   
 dob                         | 2000-10-27 00:00:00  
 height_cm                   | 178                  
 weight_kg                   | 66                   
 club_team_id                | 143.0                
 club_name                   | Exeter City          
 league_name                 | English League Two   
 league_level                | 4                    
 club_position               | RES            

In [36]:
#save the csv if you need
#df.repartition(1).write.csv("./fifa.csv",header=True)

### 4.2 Data Engineering
#### Drop columns 
the following descriptions referenced to the work of Yiren Zhou <<yirenzho@andrew.cmu.edu>>, thanks very much.\
https://github.com/ML-Systems-and-Toolchains/course-project-option-1-sample-2

I have decided to drop the following columns as they don't seem relevant to the overall value of a player:

player_url (not relevant)\
short_name (not relevant)\
long_name (not relevant) \
club_name (too many values, could be useless) \
club_loaned_from (too few values) \
club_joined(not relevant)\
dob(not relevant)\
release_clause_eur (too few values) \
player_positions (cannot be converted into numerics easily) \
goalkeeping_speed (too few values) \
player_traits (too random) \
player_tags (there are missing entries for certain players)\
real_face (not relevant) \
mentality_composure (too few values) \
player_face_url (not relevant) \
club_logo_url (not relevant) \
club_flag_url (not relevant) \
nation_logo_url (not relevant)\
nation_flag_url (not relevant)\
ls (not relevant)\
.\
.\
.\
gk(not relevant)

In [21]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset.withColumn("outcome", col("overall").cast(DoubleType()))
        output_df = output_df.drop('overall')
        return output_df
    
class NULLDropper(Transformer): # Drop all rows with NULL values.
    def __init__(self,cols):
        super().__init__()
        self.cols=cols
    
    def _transform(self, dataset):
        out_df = dataset
        return out_df.na.drop(subset=self.cols)

class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df

def get_preprocess_pipeline(df):
    
    col_names = df.columns
        
    # Drop Null columns in what we want
    nominal_cols = ["preferred_foot", "body_type", "league_name", "club_position", "nationality_name", "nation_position", "work_rate"]
    stage_dropnull = NULLDropper(nominal_cols)
    
    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_cols]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
    stage_nominal_indexer = StringIndexer(inputCols = nominal_cols, outputCols = nominal_id_cols )

    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

    # Stage where all relevant features are assembled into a vector 
    # (and dropping a few)
    feature_cols = col_names + nominal_onehot_cols
    corelated_cols_to_remove = ['player_url', 'player_positions', 'club_name', 'short_name', 'long_name', 
                                'club_loaned_from', 'club_joined', 'dob', 'release_clause_eur', 'goalkeeping_speed', 
                                'player_traits', 'player_tags', 'real_face', 'mentality_composure', 'player_face_url', 
                                'club_logo_url', 'club_flag_url', 'nation_logo_url', 'nation_flag_url',
                                    'ls','st','rs','lw','lf','cf','rf','rw','lam','cam',
                                    'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
                                    'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'
                               ]
    for col_name in nominal_cols + corelated_cols_to_remove:
        feature_cols.remove(col_name)
    stage_dropnull_afterfeature = NULLDropper(feature_cols)    
    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')
    

    # Stage for creating the outcome column representing whether there is attack 
    stage_outcome = OutcomeCreater()

    # Removing all unnecessary columbs, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = col_names+nominal_id_cols+nominal_onehot_cols+['vectorized_features','year'])
    
    
    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[stage_dropnull,stage_nominal_indexer,stage_nominal_onehot_encoder,
        stage_dropnull_afterfeature,stage_vector_assembler,stage_scaler,stage_outcome,stage_column_dropper])
    
    return pipeline

In [22]:
pipeline = get_preprocess_pipeline(df)
pipeline_model = pipeline.fit(df)
df_transformed = pipeline_model.transform(df)


# As you see, only column features and outcome are left.
df_transformed.printSchema()
df_transformed.show(1)

# Train/test split
df_train, df_test = df_transformed.randomSplit(weights=[0.8,0.2], seed=2022)

root
 |-- features: vector (nullable = true)
 |-- outcome: double (nullable = true)

+--------------------+-------+
|            features|outcome|
+--------------------+-------+
|(232,[0,1,2,3,4,5...|   53.0|
+--------------------+-------+
only showing top 1 row



### 4.3 model on spark

I choosed LinearRegression and RandomForestRegressor.

since it's a regression task, I choose MSE(mean squared error) as the target.

the dataset is devided to train(80%),test(20%)

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

evaluator = RegressionEvaluator(labelCol='outcome', metricName='mse')

#### 4.3.1 (spark) LinearRegression

"regParam" is regularization parameter, which influence the regularization process and thus contributes to model performance. The default value is '0.0'

"maxIter" is number of iterations, which determines the training times and convergence of the model. The defualt value is '100'

In [24]:
lr = LinearRegression(featuresCol='features',labelCol='outcome')

lr_param_grid = (ParamGridBuilder()
                .addGrid(lr.regParam, [0.001,0.01,0.1])
                .addGrid(lr.maxIter, [50,100,150])
                .build())
                 
lr_cv = CrossValidator(estimator=lr, 
                       estimatorParamMaps=lr_param_grid,
                       evaluator=evaluator,
                       parallelism=8,
                       numFolds=5)
                 
lr_cv_model = lr_cv.fit(df_train)

22/11/29 04:11:45 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/11/29 04:11:45 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/11/29 04:11:45 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [25]:
# after Cross-Validation

print("# after Cross-Validation")

print("Parameter: Mean Squard Error(MSE)")

lr_train_predictions = lr_cv_model.transform(df_train)
print('Train MSE of LinearRegression Model :', evaluator.evaluate(lr_train_predictions))

lr_test_predictions = lr_cv_model.transform(df_test)
print('Test MSE of LinearRegression Model :', evaluator.evaluate(lr_test_predictions))

# after Cross-Validation
Parameter: Mean Squard Error(MSE)


Train MSE of LinearRegression Model : 2.428514255680475e-05


Test MSE of LinearRegression Model : 2.4940901458240294e-05


#### 4.3.2 (spark) RandomForestRegressor
"maxDepth" means the maxmum depth of trees in the RF model, the default value is 4

"numTrees" means the number of trees in the RF model,

In [26]:
rf = RandomForestRegressor(featuresCol='features',labelCol='outcome')

rf_param_grid = (ParamGridBuilder()
                 .addGrid(rf.maxDepth, [5,10,15])
                 .addGrid(rf.numTrees, [15,20,25])
                 .build())

rf_cv = CrossValidator(estimator=rf, 
                       estimatorParamMaps=rf_param_grid,
                       evaluator=evaluator,
                       parallelism=8,
                       numFolds=5)

rf_cv_model = rf_cv.fit(df_train)

22/11/29 04:12:30 WARN DAGScheduler: Broadcasting large task binary with size 1096.9 KiB
22/11/29 04:12:30 WARN DAGScheduler: Broadcasting large task binary with size 1088.4 KiB
22/11/29 04:12:30 WARN DAGScheduler: Broadcasting large task binary with size 1088.4 KiB
22/11/29 04:12:31 WARN DAGScheduler: Broadcasting large task binary with size 1322.6 KiB
22/11/29 04:12:31 WARN DAGScheduler: Broadcasting large task binary with size 1322.6 KiB
22/11/29 04:12:31 WARN DAGScheduler: Broadcasting large task binary with size 1403.0 KiB
22/11/29 04:12:31 WARN DAGScheduler: Broadcasting large task binary with size 1403.0 KiB
22/11/29 04:12:31 WARN DAGScheduler: Broadcasting large task binary with size 1598.6 KiB
22/11/29 04:12:31 WARN DAGScheduler: Broadcasting large task binary with size 1724.6 KiB
22/11/29 04:12:31 WARN DAGScheduler: Broadcasting large task binary with size 1724.6 KiB
22/11/29 04:12:32 WARN DAGScheduler: Broadcasting large task binary with size 1649.6 KiB
22/11/29 04:12:32 WAR

22/11/29 04:12:40 WARN BlockManager: Block rdd_2526_0 already exists on this machine; not re-adding it


22/11/29 04:12:41 WARN DAGScheduler: Broadcasting large task binary with size 1078.1 KiB
22/11/29 04:12:41 WARN DAGScheduler: Broadcasting large task binary with size 1078.1 KiB
22/11/29 04:12:42 WARN DAGScheduler: Broadcasting large task binary with size 1096.6 KiB
22/11/29 04:12:42 WARN DAGScheduler: Broadcasting large task binary with size 1323.5 KiB
22/11/29 04:12:42 WARN DAGScheduler: Broadcasting large task binary with size 1323.5 KiB
22/11/29 04:12:42 WARN DAGScheduler: Broadcasting large task binary with size 1372.0 KiB
22/11/29 04:12:42 WARN DAGScheduler: Broadcasting large task binary with size 1372.0 KiB
22/11/29 04:12:42 WARN DAGScheduler: Broadcasting large task binary with size 1601.0 KiB
22/11/29 04:12:43 WARN DAGScheduler: Broadcasting large task binary with size 1606.8 KiB
22/11/29 04:12:43 WARN DAGScheduler: Broadcasting large task binary with size 1734.7 KiB
22/11/29 04:12:43 WARN DAGScheduler: Broadcasting large task binary with size 1606.8 KiB
22/11/29 04:12:43 WAR

22/11/29 04:12:53 WARN DAGScheduler: Broadcasting large task binary with size 1082.1 KiB
22/11/29 04:12:53 WARN DAGScheduler: Broadcasting large task binary with size 1075.7 KiB
22/11/29 04:12:53 WARN DAGScheduler: Broadcasting large task binary with size 1075.7 KiB
22/11/29 04:12:53 WARN DAGScheduler: Broadcasting large task binary with size 1335.4 KiB
22/11/29 04:12:53 WARN DAGScheduler: Broadcasting large task binary with size 1335.4 KiB
22/11/29 04:12:54 WARN DAGScheduler: Broadcasting large task binary with size 1365.6 KiB
22/11/29 04:12:54 WARN DAGScheduler: Broadcasting large task binary with size 1365.6 KiB
22/11/29 04:12:54 WARN DAGScheduler: Broadcasting large task binary with size 1575.0 KiB
22/11/29 04:12:54 WARN DAGScheduler: Broadcasting large task binary with size 1744.5 KiB
22/11/29 04:12:54 WARN DAGScheduler: Broadcasting large task binary with size 1744.5 KiB
22/11/29 04:12:54 WARN DAGScheduler: Broadcasting large task binary with size 1601.6 KiB
22/11/29 04:12:55 WAR

22/11/29 04:13:04 WARN DAGScheduler: Broadcasting large task binary with size 1083.4 KiB
22/11/29 04:13:04 WARN DAGScheduler: Broadcasting large task binary with size 1076.1 KiB
22/11/29 04:13:04 WARN DAGScheduler: Broadcasting large task binary with size 1076.1 KiB
22/11/29 04:13:05 WARN DAGScheduler: Broadcasting large task binary with size 1332.9 KiB
22/11/29 04:13:05 WARN DAGScheduler: Broadcasting large task binary with size 1332.9 KiB
22/11/29 04:13:05 WARN DAGScheduler: Broadcasting large task binary with size 1380.6 KiB
22/11/29 04:13:05 WARN DAGScheduler: Broadcasting large task binary with size 1380.6 KiB
22/11/29 04:13:05 WARN DAGScheduler: Broadcasting large task binary with size 1583.1 KiB
22/11/29 04:13:06 WARN DAGScheduler: Broadcasting large task binary with size 1729.5 KiB
22/11/29 04:13:06 WARN DAGScheduler: Broadcasting large task binary with size 1729.5 KiB
22/11/29 04:13:06 WARN DAGScheduler: Broadcasting large task binary with size 1618.9 KiB
22/11/29 04:13:06 WAR

22/11/29 04:13:16 WARN DAGScheduler: Broadcasting large task binary with size 1065.4 KiB
22/11/29 04:13:16 WARN DAGScheduler: Broadcasting large task binary with size 1090.7 KiB
22/11/29 04:13:16 WARN DAGScheduler: Broadcasting large task binary with size 1065.4 KiB
22/11/29 04:13:16 WARN DAGScheduler: Broadcasting large task binary with size 1303.2 KiB
22/11/29 04:13:16 WARN DAGScheduler: Broadcasting large task binary with size 1303.2 KiB
22/11/29 04:13:16 WARN DAGScheduler: Broadcasting large task binary with size 1360.0 KiB
22/11/29 04:13:16 WARN DAGScheduler: Broadcasting large task binary with size 1360.0 KiB
22/11/29 04:13:17 WARN DAGScheduler: Broadcasting large task binary with size 1584.9 KiB
22/11/29 04:13:17 WARN DAGScheduler: Broadcasting large task binary with size 1696.5 KiB
22/11/29 04:13:17 WARN DAGScheduler: Broadcasting large task binary with size 1696.5 KiB
22/11/29 04:13:17 WARN DAGScheduler: Broadcasting large task binary with size 1602.7 KiB
22/11/29 04:13:17 WAR

22/11/29 04:13:29 WARN DAGScheduler: Broadcasting large task binary with size 1090.5 KiB
22/11/29 04:13:30 WARN DAGScheduler: Broadcasting large task binary with size 1633.6 KiB
22/11/29 04:13:31 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/11/29 04:13:32 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/11/29 04:13:33 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
22/11/29 04:13:33 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/11/29 04:13:34 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
22/11/29 04:13:34 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
22/11/29 04:13:35 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


In [27]:
# after Cross-Validation

print("# after Cross-Validation")

rf_train_predictions = rf_cv_model.transform(df_train)
print('Train MSE of RandomForest Model is :', evaluator.evaluate(rf_train_predictions))

rf_test_predictions = rf_cv_model.transform(df_test)
print('Test MSE of RandomForest Model is :', evaluator.evaluate(rf_test_predictions))

# after Cross-Validation


Train MSE of RandomForest Model is : 0.014993017564290159


Test MSE of RandomForest Model is : 0.08411291366062913


### 4.4 Model on Tensorflow
I built Basic regression Model (only 1 layer, which is like Linear Regression) and Deep Nueral Network(more than 3 layers)

since it's a regression task, I choose MSE(mean squared error) as the target, which is also the loss I used.

the dataset is devided to train(80%),validate(10%),test(10%)

In [30]:
import tensorflow as tf
from tensorflow import keras

to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

df_test_half, df_validate = df_test.randomSplit(weights=[0.5, 0.5], seed=2022)

df_train_pandas = df_train.withColumn('features', to_array('features')).toPandas()
df_validate_pandas = df_validate.withColumn('features', to_array('features')).toPandas()
df_test_half_pandas = df_test_half.withColumn('features', to_array('features')).toPandas()

x_train = tf.constant(np.array(df_train_pandas['features'].values.tolist()))
y_train = tf.constant(np.array(df_train_pandas['outcome'].values.tolist()))

x_validate = tf.constant(np.array(df_validate_pandas['features'].values.tolist()))
y_validate = tf.constant(np.array(df_validate_pandas['outcome'].values.tolist()))

x_test = tf.constant(np.array(df_test_half_pandas['features'].values.tolist()))
y_test = tf.constant(np.array(df_test_half_pandas['outcome'].values.tolist()))

#### 4.4.1 (tensorflow) Basic regression 

the Basic regression model only has 1 layer, and choosed the loss of 'mean_squared_error'

the hyperpamameter is:\
learningrate = [0.01,0.1]\
epochs = [10,100]

In [31]:
#to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

#df_transformed_pandas = df_transformed.withColumn('features', to_array('features')).toPandas()

HP_learningrate = [0.01,0.1]
HP_epochs = [10,100]

learningrate = []
epochs = []
loss = []
val_loss = []

for r in HP_learningrate:
    for e in HP_epochs:
        model = keras.Sequential()
        model.add(keras.layers.Dense(1))   
        model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(r))

        history = model.fit(x_train, y_train, epochs = e, verbose = 0,
                            validation_data = (x_validate, y_validate))
        loss.append(np.min(history.history["loss"]))
        val_loss.append(np.min(history.history["val_loss"]))
        learningrate.append(r)
        epochs.append(e)


        print('--- learningrate is :',r)
        print('--- epochs is :',e)
        print('--- loss(mean_squared_error,mse) for this trial is:',loss[-1])
        print('--- val_loss(mean_squared_error,mse) for this trial is:',val_loss[-1])
        print()

--- learningrate is : 0.01
--- epochs is : 10
--- loss(mean_squared_error,mse) for this trial is: 3.7285938262939453
--- val_loss(mean_squared_error,mse) for this trial is: 2.725465774536133

--- learningrate is : 0.01
--- epochs is : 100
--- loss(mean_squared_error,mse) for this trial is: 1.0989384651184082
--- val_loss(mean_squared_error,mse) for this trial is: 0.6596671938896179

--- learningrate is : 0.1
--- epochs is : 10
--- loss(mean_squared_error,mse) for this trial is: 6.85516881942749
--- val_loss(mean_squared_error,mse) for this trial is: 5.283373832702637

--- learningrate is : 0.1
--- epochs is : 100
--- loss(mean_squared_error,mse) for this trial is: 5.850796222686768
--- val_loss(mean_squared_error,mse) for this trial is: 2.210355043411255



In [32]:
best_lr = learningrate[np.array(val_loss).argmin()]
best_epochs = epochs[np.array(val_loss).argmin()]

best_model = keras.Sequential()
best_model.add(keras.layers.Dense(1))

best_model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(best_lr))

history = best_model.fit(x_train,y_train, epochs = best_epochs,validation_data=(x_validate,y_validate),verbose = 0)
a = np.min(history.history["loss"])
b = np.min(history.history["val_loss"])
c = best_model.evaluate(x_test,y_test, verbose = 2)

print()
print('best model has learningrate of', best_lr)
print('best model has epochs of', best_epochs)
print("train loss (mean squared error, MSE) of best Model is :",a)
print("validation loss (mean squared error, MSE) of best Model is :",b)
print("test loss (mean squared error, MSE) of best Model is :",c)

20/20 - 0s - loss: 4.8586 - 29ms/epoch - 1ms/step

best model has learningrate of 0.01
best model has epochs of 100
train loss (mean squared error, MSE) of best Model is : 0.9805044531822205
validation loss (mean squared error, MSE) of best Model is : 0.6801302433013916
test loss (mean squared error, MSE) of best Model is : 4.858574390411377


#### 4.4.2 (tensorflow) Deep Nueral Network
the Deep Nueral Network model has more than 3 layers, and choosed the loss of 'mean_squared_error', the activation function of 'relu',epochs = 100

the hyperpamameter is:\
WIDTH = [10,15,20]\
DEPTH = [3,5,7]

In [34]:
#to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

#df_transformed_pandas = df_transformed.withColumn('features', to_array('features')).toPandas()

HP_WIDTH = [10,15,20]
HP_DEPTH = [3,5,7]

depth = []
width = []
loss = []
val_loss = []

for dep in HP_DEPTH:
    for wid in HP_WIDTH:
        model = keras.Sequential()
        for i in range(dep):
            model.add(keras.layers.Dense(wid,activation='relu'))
        model.add(keras.layers.Dense(1))   
        model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(0.01))

        history = model.fit(x_train, y_train, epochs = 100, verbose = 0,
                            validation_data = (x_validate, y_validate))
        loss.append(np.min(history.history["loss"]))
        val_loss.append(np.min(history.history["val_loss"]))
        depth.append(dep)
        width.append(wid)


        print('--- Depth is :',dep)
        print('--- Width is :',wid)
        print('--- loss(mean_squared_error,mse) for this trial is:',loss[-1])
        print('--- val_loss(mean_squared_error,mse) for this trial is:',val_loss[-1])
        print()

--- Depth is : 3
--- Width is : 10
--- loss(mean_squared_error,mse) for this trial is: 2.1048104763031006
--- val_loss(mean_squared_error,mse) for this trial is: 1.4913729429244995

--- Depth is : 3
--- Width is : 15
--- loss(mean_squared_error,mse) for this trial is: 2.1714603900909424
--- val_loss(mean_squared_error,mse) for this trial is: 1.5114251375198364

--- Depth is : 3
--- Width is : 20
--- loss(mean_squared_error,mse) for this trial is: 2.256932258605957
--- val_loss(mean_squared_error,mse) for this trial is: 1.5276556015014648

--- Depth is : 5
--- Width is : 10
--- loss(mean_squared_error,mse) for this trial is: 2.178896903991699
--- val_loss(mean_squared_error,mse) for this trial is: 1.6898306608200073

--- Depth is : 5
--- Width is : 15
--- loss(mean_squared_error,mse) for this trial is: 1.8955179452896118
--- val_loss(mean_squared_error,mse) for this trial is: 1.2388601303100586

--- Depth is : 5
--- Width is : 20
--- loss(mean_squared_error,mse) for this trial is: 1.682

In [35]:
best_wid = width[np.array(val_loss).argmin()]
best_dep = depth[np.array(val_loss).argmin()]

best_model = keras.Sequential()
for i in range(best_dep):
    best_model.add(keras.layers.Dense(best_wid,activation='relu'))
best_model.add(keras.layers.Dense(1))

best_model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(0.01))

history = best_model.fit(x_train,y_train, epochs = 100,validation_data=(x_validate,y_validate),verbose = 0)
a = np.min(history.history["loss"])
b = np.min(history.history["val_loss"])
c = best_model.evaluate(x_test,y_test, verbose = 2)

print()
print('best model has depth of', best_dep)
print('best model has width of', best_wid)
print("train loss (mean squared error, MSE) of best Model is :",a)
print("validation loss (mean squared error, MSE) of best Model is :",b)
print("test loss (mean squared error, MSE) of best Model is :",c)

20/20 - 0s - loss: 1.8554 - 30ms/epoch - 1ms/step

best model has depth of 7
best model has width of 20
train loss (mean squared error, MSE) of best Model is : 2.519286632537842
validation loss (mean squared error, MSE) of best Model is : 1.4494751691818237
test loss (mean squared error, MSE) of best Model is : 1.85536527633667
